# Preprocessing

In [1]:
# math and data packages
import pandas as pd
import numpy as np
import math

# charting and graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# os and file types
import os
import sys
import datetime as dt
import json
import csv

# images and display
import base64, io, IPython
from PIL import Image as PILImage
from IPython.display import Markdown as md
from IPython.display import display, Math, Latex


# set useful variables for accessing the files
here = os.getcwd()
flora_h = F"{here}/resources/"
flora_h_ws = F"{flora_h}atlasws/"
flora_h_55 = F"{flora_h}atlas5x5/"
data_2020 = F"{here}/resources/"

## Import data

Fix any known formatting problems here

In [2]:
# importing the data files
# create variables for convenience method and grouping

# start by organizing them into dictionaries.
my_data_methods = {"csv":pd.read_csv}

# dict for the 2020 survey data files
d_files = {
    "surveys_20":"surveys.csv",
    "surveys_21a":"data-2021.csv",
    "surveys_21b":"obs_export_2021-10-26_20h46.csv",
    "map_keys_20":"map-keys.csv",
    "map_keys_21":"2021-survey-key.csv"
}
# match map_keys_21 to surveys_21A
# date and time format correction
# species slug
# format to 2020

# dict for the reference files that are not Welten Sutter or Flora Helvetica 5X5 lists
w_lists = {
    "list_2014":"BL_WL_2014_modified.csv",
    "under_sampled":"taxa_sous_echantillonnes.csv",
    "red_list":"redlist2019.csv",
    "cert_list": "Certification_specieslist_2021.csv",
    "ch_priority":"ch_priority_species.csv"
}

# format and import ch_priority_species.csv to use with 2020 & 2021 survey results
# evaluate the survey results with respect to the columns "priorité	menace	responsabilité	nécessité de prendre des mesures	nécessité de surveiller les populations	connaissances suffisantes?	techniques connues?
# repeat 2020 data with redlist with 2021A

v_lists = {
    "imp_seeds":"important-seeds-2021.csv",
    "all_seeds":"all-seeds-2021.csv",
    "inv_cave":"volo-inventory-de.csv",
    "germ_exp":"germination-experiment.csv"
    
}
# add dict for the volo files

# dict for the Welten-Sutter map reference files, downloaded from here: https://www.infoflora.ch/de/daten/artenliste-welten-sutter.html
# all observations included in this report were conducted within one of these geographic boundaries
ws_lists ={
    "151":"AtlasWS_151_Biel.csv",
    "252":"AtlasWS_252_Erlach.csv",
    "300":"AtlasWS_300_Aarberg.csv",
    "301":"AtlasWS_301_Bueren.csv",
    "154":"AtlasWS_154_Grenchen.csv",
    "572":"AtlasWS_572_Beatenberg.csv",
    "573":"AtlasWS_573_Interlaken.csv",
    "226":"AtlasWS_226_Estavayer.csv",
    "251":"AtlasWS_251_BernWest.csv",
    "145":"AtlasWS_145_LesRangiers.csv"
}

# housekeeping: 585220 is separated by "," not ";" like the rest of the data sources
df = pd.read_csv("resources/atlas5x5/Atlas5x5_585_220.csv", sep = ",", encoding="utf-16")
df.to_csv('resources/atlas5x5/Atlas5x5_585_220_1.csv', sep=';', encoding = "utf-16", index = False)

# 5X5 Flora helvitca lists
fx_lists = {
    "585215":"Atlas5x5_585_215.csv", # Ipsach, Bielersee
    "585220":"Atlas5x5_585_220_1.csv", # Biel Stadt, Suze / Bielersee
    "580220":"Atlas5x5_580_220.csv", # Biel Mett, Suze
    "580215":"Atlas5x5_580_215.csv", # Port, Nidau-Bueren Kanal
    "625165":"Atlas5x5_625_165.csv", # Untersee, Thunersee
    "625170":"Atlas5x5_625_170.csv", # Sundlauenen, Thunersee
    "550185":"Atlas5x5_550_185.csv", # Estavayer, Lac de Neuchatel
    "575210":"Atlas5x5_575_210.csv", # Leuecherz, Bielersee
    "600200":"Atlas5x5_600_200.csv", # Bern west, Aare
    "575245":"Atlas5x5_575_245.csv", # Saint-Ursanne, Aare
    "545180":"Atlas5x5_545_180.csv",
    "575215":"Atlas5x5_575_215.csv"
}

# convenience method to gather up all the files:
def get_the_data(file_exts, a_dir, methods, this_method="csv", myencoding=None):
    wiw = {}
    for k,v in file_exts.items():
        if myencoding == None:
            wiw.update({k:methods[this_method](F"{a_dir}{v}")})            
        else:
            wiw.update({k:methods[this_method](F"{a_dir}{v}",sep = ";", encoding=myencoding)})
    return wiw

# use the get_the_data method to collect these files
data_and_keys = get_the_data(d_files, data_2020, my_data_methods, this_method="csv")
watch_lists = get_the_data(w_lists, flora_h, my_data_methods, this_method="csv")

# Why are we usig utf-16 here? Don't we lose some options later on?
welt_sut =  get_the_data(ws_lists, flora_h_ws, my_data_methods, this_method="csv", myencoding = "utf-16" )
fivex =  get_the_data(fx_lists, flora_h_55, my_data_methods, this_method="csv", myencoding = "utf-16")

FileNotFoundError: [Errno 2] No such file or directory: '/home/mwshovel/dev/collaborate/botany/florabuckets/flora-buckets/resources/data-2021.csv'

## The Species columns

In [ ]:
# make sure that each data set has the column "species", with the value species:
watch_lists["list_2014"]["species"] = watch_lists["list_2014"].Latin
watch_lists["under_sampled"]["species"] = watch_lists["under_sampled"].taxon
watch_lists["red_list"]["species"] = watch_lists["red_list"].scientific_name
watch_lists["cert_list"]["species"] = watch_lists["cert_list"]["Short Name"]

In [ ]:
# make a species slug (genus-species) to link data from across the survey and reference files. 
# This is necessary as some species columns have only "Genus species", some include subspecies, and some include the taxonomic reference.

# function to make the species slugs
def to_species_slug(x):
    try: 
        int_data = x.split()
        data = int_data[:2]
        data = "-".join(data)
        data = data.lower()
    except:
        data = "none"
    return data

# create a new column to hold the slug
for element in [fivex, welt_sut, watch_lists]:
    for the_data in element:
        element[the_data]['species_slug'] = 'none'

# make the species slug for all reference files
for element in [fivex, welt_sut, watch_lists]:
    for the_data in element:
        element[the_data]['species_slug'] = element[the_data].species.map(lambda x: to_species_slug(x))

## The _map_ column and the _spec\_map_ columns

In [ ]:
# Add identifying columns to the reference datasets

# add a column to identify the map source for the geographic data:
for element in [fivex, welt_sut]:
    for the_data in element:
        element[the_data]['map'] = the_data
        element[the_data]['spec_map'] = list(zip(element[the_data].species_slug,element[the_data].map))
        


In [ ]:
fivex.keys()

In [ ]:
fivex['585215'].head()

## The _watch\_list_ column

In [ ]:
for element in [watch_lists]:
    for the_data in element:
        if the_data == "list_2014":
            element[the_data]['watch_list'] = element[the_data][the_data]
        else:
            element[the_data]['watch_list'] = the_data

# housekeeping: fill in nan values in the watchlist and certification list reference files.
fill_nans = watch_lists["list_2014"].copy()
fill_nans = fill_nans.fillna(0)
watch_lists.update({"list_2014":fill_nans[fill_nans.watch_list != 0]})

fill_nans = watch_lists["cert_list"].copy()
fill_nans = fill_nans.fillna(0)
watch_lists.update({"cert_list":fill_nans[fill_nans.watch_list != 0]})

In [ ]:
watch_lists.keys()

In [ ]:
watch_lists['red_list'].head()

### Species name and observations: harmonizing taxonomy

The genus-species nomenclature will be used to group observations.

All observations will be classified according to that standard. As a result subspecies will be folded in with the parent species. This is a reflection of the survey method and the expectation of reasonable results, not a prioritization of importance.

In [ ]:
def account_for_subspecies(an_array, a_dict):
    for element in an_array:
        try:
            a_dict[element[0]].append(element[1])
        except:
            a_dict[element[0]] = [element[1]]
    return a_dict
a_dict ={}

for element in [fivex, welt_sut, watch_lists]:
    for label in element:
        # use this data frame
        som_data = element[label].copy()
        
        # group by species slug and count the number of unique species values
        c_s_p_s = som_data.groupby('species_slug', as_index=False).species.nunique()        
        
        # just the records with more than one species value
        m_t_one = c_s_p_s[c_s_p_s.species > 1].species_slug
        
        # pair the species_slug to the species name:
        mto = som_data.loc[som_data.species_slug.isin(m_t_one)][['species_slug', 'species']].copy().to_numpy()
        
        # update the dict
        account_for_subspecies(mto, a_dict)

# the species_slugs that account for more than one sub species
sub_species_accounted =  {k:list(set(v)) for k,v in a_dict.items()}

# the species_slugs
gs_parent = list(sub_species_accounted.keys())

# the number of species_slugs
number_of_gs = len(gs_parent)

# the the number of sub species accounted for
number_of_ss = sum([len(v) for k,v in sub_species_accounted.items()])

## Determine wether or not a species was detected within a geographic limit

The territory is divided into different segments. Flora-helvitica and WS maps have different geographic bounds. Here the presence or not of a species within the confines of one of the different boundaries is determined.

In [ ]:
# collect all the observations from flora helvitaca and the WS into one df
fx = pd.concat([v[['species_slug', 'map', 'spec_map']] for k,v in fivex.items()])
wsx = pd.concat([v[['species_slug', 'map', 'spec_map']] for k,v in welt_sut.items()])
f_w_obs = pd.concat([fx, wsx])

In [ ]:
f_w_obs.head()

In [ ]:
# replacedict = {
#     'verbanum bonariensis ':'verbena bonariensis',
#     'medicago varia':'medicago sativa',
#     "oenothera":"oenothera biennis",
#     "geranium pratens":"geranium pratense",
#     "oenothera biennis ": "oenothera biennis",
#     "oenothera biennis agg.": "oenothera biennis",
#     "solidalgo canadensis": "solidago canadensis",
#     "verbascum lynchitis":"verbascum lychnitis",
#     "verbascum negris":"verbascum nigrum",
#     "securigea varia": "securigera varia",
#     "melilotus officianalis": "melilotus officinalis",
#     "knautia maxima": "knautia dipsacifolia",
#     "hieracium aurantiacum":"pilosella aurantiaca",
#     "sysimbrium officinale":"sisymbrium officinale",
#     "geranium robertanium":"geranium robertianum",
#     "mycelis muralis": "lactuca muralis",
#     "calamintha-nepeta":"clinopodium nepeta",
#     "polygonum-persicaria":"persicaria maculosa",
#     "sorbus-aria":"aria edulis",
#     "taraxacum": "taraxacum officinale",
#     "jacobaea vulgaris" : "senecio jacobaea",
#     "erigeron canadensis" : "conyza canadensis",
#     "rorippa islandica" : "rorippa palustris",
#     "malus sylvestris" : "malus domestica",
#     "hylotelephium telephium" : "sedum telephium",
#     "lactuca muralis": "mycelis muralis",
#     "chaenorhinum minus": "chaenorrhinum minus",
#     "erigeron canadensis": "conzya canadensis",
#     "erigeron canadensis": "conzya canadensis",
#     "borkhausenia intermedia": "scandosorbus intermedia",
#     "centaurea nigra" : "centaurea jacea"
# }

In [ ]:
# f_w_obs['ns'] = f_w_obs.species_slug
# def replace_this(x,a_dict):
#     if x in a_dict.keys():
#         data=a_dict[x]
#     else:
#         data = x
#     return data
# f_w_obs['ns'] = f_w_obs.ns.map(lambda x: replace_this(x, replacedict))
# f_w_obs['species_slug'] = f_w_obs.ns

In [ ]:
astring = F"""
There are {len(f_w_obs.map.unique())} different map boundaries in this study
"""
md(astring)

In [ ]:
# a = f_w_obs.set_index('species_slug')
# a.loc['bryonia-dioica']

In [ ]:
# gather up the map names
m_ap_columns = f_w_obs.map.unique()

# create a column for each map, indicate
for col in m_ap_columns:
    
    f_w_obs[col] = f_w_obs['map'] == col

obs_map =f_w_obs.groupby(['species_slug']).sum()

# human readable column names need to be introduced here or a dict to rename
obs_map.head()

In [ ]:
obs_map.loc['bryonia-dioica']

In [ ]:
a_file_name = "species_map_located.csv"
obs_map.to_csv(F"resources/survey-data/{a_file_name}", index=True)

### Key the species to the different maps it was identified in


In [ ]:
# #exports the dictionary to a .json file
# nt = obs_map['maps'].to_dict()

# ntx = {k:list(v) for k,v in nt.items()}

# with open(F"{here}/output/ws_list.json","w") as afile:
#     json.dump(ntx,afile)
    
# print(F"\nWhich maps 'trifolium-incarnatum' were found in? indifferent of subspecies?:\n\n{ntx['trifolium-incarnatum']}\n")

## The _species\_slug_ column

A coder friendly way to find the species but still maintain the proper nomenclature

In [ ]:
# format scientific name in the sample dataset
samples = data_and_keys['surveys'].copy()

samples.head()

In [ ]:
# dictionary of replacement values that are incorrect
replacedict = {
    'verbanum bonariensis ':'verbena bonariensis',
    'medicago varia':'medicago sativa',
    "oenothera":"oenothera biennis",
    "geranium pratens":"geranium pratense",
    "oenothera biennis ": "oenothera biennis",
    "oenothera biennis agg.": "oenothera biennis",
    "solidalgo canadensis": "solidago canadensis",
    "verbascum lynchitis":"verbascum lychnitis",
    "verbascum negris":"verbascum nigrum",
    "securigea varia": "securigera varia",
    "melilotus officianalis": "melilotus officinalis",
    "knautia maxima": "knautia dipsacifolia",
    "hieracium aurantiacum":"pilosella aurantiaca",
    "sysimbrium officinale":"sisymbrium officinale",
    "geranium robertanium":"geranium robertianum",
    "mycelis muralis": "lactuca muralis",
    "calamintha-nepeta":"clinopodium nepeta",
    "polygonum-persicaria":"persicaria maculosa",
    "sorbus-aria":"aria edulis",
    "taraxacum": "taraxacum officinale",
    "jacobaea vulgaris" : "senecio jacobaea",
    "erigeron canadensis" : "conyza canadensis",
    "rorippa islandica" : "rorippa palustris",
    "malus sylvestris" : "malus domestica",
    "hylotelephium telephium" : "sedum telephium",
    "lactuca muralis": "mycelis muralis",
    "chaenorhinum minus": "chaenorrhinum minus",
    "erigeron canadensis": "conzya canadensis",
    "erigeron canadensis": "conzya canadensis",
    "borkhausenia intermedia": "scandosorbus intermedia",
    "centaurea nigra" : "centaurea jacea"
}

# function to assign the correct value of the key is in the samples dictionary.
def new_func(x,keys):
    try:
        data = keys[x]
    except:
        data = x
    return data

# apply the funtion to a copy of the surveys data set.
samples["species2"] = samples.sci.map(lambda x: new_func(x, replacedict))
samples["species_slug"] = samples.species2.map(lambda x: to_species_slug(x))

# update the surveys dataset.
data_and_keys.update({'surveys':samples})

In [ ]:
data_and_keys["surveys"]

## Format date column to ISO standard

In [ ]:
# format dates in the sample dataset

# function converts DD.MM.YYYY format to YYYY-MM-DD format, ignores if already in YYYY-MM-DD format
# Do we even need this any more ? Looking at the cell above it appears the dates have been fixed at the source
def change_string(x):
    try:
        s_data = x.split('.')
        data = s_data[::-1]
        data = "-".join(data)
    except:
        print("no luck")
        data = x
    
    return data

# applies the function to a column in the samples data frame
samples['new_date'] = samples.date.map(lambda x: change_string(x))

# function makes a timestamp out of the YYYY-MM-DD string.
# def make_timestamp(x):
#     try:        
#         data = dt.datetime.strptime(x, "%Y-%m-%d")        
#     except:        
#         data = 'no luck'
    
#     return data

# run the make_timestamp function and store the results in the samples dataframe.
# samples['stamp_date'] = samples.new_date.map(lambda x: make_timestamp(x))
# samples['date'] = samples.stamp_date

## The _place1_ column

A coder friendly way to find the species but still maintain the proper nomenclature

In [ ]:
# make place name slugs in the sample dataset
#  change_place function turns "place names 1" into place-names-1
def change_place(x):
    data = x.split(" ")
    data = "-".join(data)
    return data

samples["place_slug"] = samples.place.map(lambda x: change_place(x))

## Housekeeping and export of processed survey data

In [ ]:
# clean up redundant and unused column names
# rename the place_slug column

samples.rename(columns= {'place1':'place_slug', 'new_date':'str_date'}, inplace=True)

In [ ]:
samples.columns

In [ ]:
# remove the unnecessary columns for this analysis
samples.drop(['time',  'sci', 'name', 'place', 'species2'] , inplace=True, axis=1)

samples['loc_date'] = list(zip(samples.place_slug, samples.date))

In [ ]:
# export samples to a .csv for later use
samples.to_csv(F"resources/preprocessed/hd_samples_2020.csv", index=False)

In [ ]:
samples.head()

Subspecies

In [ ]:
def account_for_subspecies(an_array, a_dict):
    for element in an_array:
        try:
            a_dict[element[0]].append(element[1])
        except:
            a_dict[element[0]] = [element[1]]
    return a_dict
a_dict ={}

for element in [fivex, welt_sut, watch_lists]:
    for label in element:
        # use this data frame
        som_data = element[label].copy()
        
        # group by species slug and count the number of unique species values
        c_s_p_s = som_data.groupby('species_slug', as_index=False).species.nunique()        
        
        # just the records with more than one species value
        m_t_one = c_s_p_s[c_s_p_s.species > 1].species_slug
        
        # pair the species_slug to the species name:
        mto = som_data.loc[som_data.species_slug.isin(m_t_one)][['species_slug', 'species']].copy().to_numpy()
        
        # update the dict
        account_for_subspecies(mto, a_dict)

# the species_slugs that account for more than one sub species
sub_species_accounted =  {k:list(set(v)) for k,v in a_dict.items()}

# the species_slugs
gs_parent = list(sub_species_accounted.keys())

# the number of species_slugs
number_of_gs = len(gs_parent)

# the the number of sub species accounted for
number_of_ss = sum([len(v) for k,v in sub_species_accounted.items()])

### Key the watch lists to genus-species

Identify which species are on which watch list. Produce a boolean matrix with species_slug as index and watch list name for the columns.

In [ ]:
the_lists = list(watch_lists.keys())

keep = []

for element in the_lists:
    a = watch_lists[element][['species_slug','watch_list']]
    keep.append(a)
wl_species = pd.concat(keep)
m_ap_columns = wl_species.watch_list.unique()

for col in m_ap_columns:    
    wl_species[col] = wl_species['watch_list'] == col

wl_sp_map =wl_species.groupby(['species_slug']).sum()

wl_sp_map['lists'] = wl_sp_map.index.map(lambda x: wl_species[wl_species.species_slug == x]['watch_list'].unique())

a_filename = "species_keyed_to_watch_list.csv"
wl_sp_map.reset_index().to_csv(F"resources/{a_filename}", index=False)

In [ ]:
wl_sp_map

In [ ]:
# nt = wl_sp_map['lists'].to_dict()

# ntx = {k:list(v) for k,v in nt.items()}

# with open(F"{here}/output/flora_list.json","w") as afile:
#     json.dump(ntx,afile)
# wl_sp_map.to_csv(F"{here}/resources/preprocessed/aggregated_list.csv",index=True)